In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36827")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Lead Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc.

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
# Check if it's a list and extract the first dictionary
if isinstance(json_res, list) and len(json_res) > 0:
    json_dict = json_res[0]  # Access the first element
else:
    json_dict = json_res  # Already a dictionary or empty list
json_dict

{'role': 'Lead Software Engineer, ITC',
 'experience': '8+ years',
 'skills': ['Technical architecture',
  'Design',
  'Development',
  'Demand or inventory planning products/applications',
  'Integrated Business Planning solutions like O9',
  'PL/SQL',
  'Cloud environments such as AWS, Azure or GCP',
  'Scrum and other Agile processes',
  'Jira, Git/SVN, Jenkins, CI/CD pipelines'],
 'description': 'Lead and mentor a team of engineers with a focus to develop solutions for the various products of Demand & Supply Planning capability. Advocate and implement business transformation solutions and best practices. Advocate DevOps practices & mindset and strong culture of innovation.'}

In [17]:
type(json_dict)

dict

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
job = json_dict
job['skills']

['Technical architecture',
 'Design',
 'Development',
 'Demand or inventory planning products/applications',
 'Integrated Business Planning solutions like O9',
 'PL/SQL',
 'Cloud environments such as AWS, Azure or GCP',
 'Scrum and other Agile processes',
 'Jira, Git/SVN, Jenkins, CI/CD pipelines']

In [19]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}]]

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Taran, a business development executive at RiseAI. RiseAI is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of RiseAI 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase RiseAI's portfolio: {link_list}
        Remember you are Taran, BDE at RiseAI. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for ITC Demand & Supply Planning

Dear Hiring Manager,

I came across the job description for a Lead Software Engineer, ITC, and I'm excited to introduce RiseAI as a potential partner to help you achieve your goals. With 8+ years of experience in technical architecture, design, development, and integrated business planning solutions, our team is well-equipped to support your Demand & Supply Planning capability.

At RiseAI, we specialize in developing tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in cloud environments such as AWS, Azure, or GCP, Scrum, and Agile processes, as well as tools like Jira, Git/SVN, Jenkins, and CI/CD pipelines, aligns perfectly with your requirements.

We've empowered numerous enterprises with our innovative solutions, and I'd like to highlight a few relevant examples from our portfolio:

* Our expertise in DevOps is showcased in